# 📘 Amharic NER Fine-Tuning on Telegram Messages

# Install dependencies

In [ ]:
!pip install --quiet --no-cache-dir \
  transformers \
  datasets \
  evaluate \
  seqeval \
  accelerate \
  fsspec \
  gcsfs

# Upload the ner_dataset.conll file

In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload ner_dataset.conll

# Load CoNLL data into Hugging Face format

In [ ]:
from datasets import Dataset, DatasetDict

def read_conll_file(path):
    tokens, labels, data = [], [], []
    with open(path, encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                if tokens and len(tokens) == len(labels):
                    data.append({"tokens": tokens, "ner_tags": labels})
                else:
                    print(f"Skipping incomplete sentence: {tokens}, {labels}")
                tokens, labels = [], []
            else:
                splits = line.split()
                if len(splits) != 2:
                    print(f"Skipping invalid line: {line}")
                    continue
                tokens.append(splits[0])
                labels.append(splits[1])
    if tokens and len(tokens) == len(labels):
        data.append({"tokens": tokens, "ner_tags": labels})
    else:
        print(f"Skipping incomplete final sentence: {tokens}, {labels}")
    print(f"Loaded {len(data)} sentences from {path}")
    return data

dataset_data = read_conll_file("ner_dataset.conll")
dataset = Dataset.from_list(dataset_data).train_test_split(test_size=0.2)
raw_datasets = DatasetDict({"train": dataset["train"], "validation": dataset["test"]})

# Inspect label distribution

In [ ]:
from collections import Counter
label_counts = Counter([label for example in dataset_data for label in example["ner_tags"]])
print("Label distribution:", label_counts)

# Tokenizer and label preparation

In [ ]:
from transformers import AutoTokenizer

model_name = "xlm-roberta-base"  # Consider "distilbert-base-multilingual-cased" for smaller datasets
tokenizer = AutoTokenizer.from_pretrained(model_name)

unique_labels = sorted({label for row in dataset_data for label in row['ner_tags']})
label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for label, idx in label2id.items()}
print("Unique labels:", unique_labels)

def encode_tags(example):
    example["ner_tags"] = [label2id[tag] for tag in example["ner_tags"]]
    return example

raw_datasets = raw_datasets.map(encode_tags)

# Tokenize and align labels

In [ ]:
def tokenize_and_align_labels(example):
    tokenized_inputs = tokenizer(example["tokens"], truncation=True, is_split_into_words=True)
    word_ids = tokenized_inputs.word_ids()

    label_ids = []
    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
        elif word_idx != previous_word_idx:
            if word_idx >= len(example["ner_tags"]):
                print(f"Error: word_idx {word_idx} exceeds ner_tags length {len(example['ner_tags'])}")
                label_ids.append(-100)
            else:
                label_ids.append(example["ner_tags"][word_idx])
        else:
            label_ids.append(-100)  # Use -100 for subword tokens
        previous_word_idx = word_idx
    tokenized_inputs["labels"] = label_ids
    return tokenized_inputs

tokenized_datasets = raw_datasets.map(tokenize_and_align_labels, batched=False)

# Initialize model and training arguments

In [ ]:
from transformers import (
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
import torch

model = AutoModelForTokenClassification.from_pretrained(
    model_name, num_labels=len(label2id), id2label=id2label, label2id=label2id
)

# Compute class weights
label_counts_dict = {label: label_counts.get(label, 1) for label in unique_labels}
total = sum(label_counts_dict.values())
class_weights = torch.tensor([total / (len(unique_labels) * label_counts_dict[label]) for label in unique_labels]).to("cuda" if torch.cuda.is_available() else "cpu")
print("Class weights:", class_weights)

training_args = TrainingArguments(
    output_dir="./amharic-ner-model",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=1,
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    run_name="amharic-ner-run",
    report_to="none"  # Remove if using W&B
)

data_collator = DataCollatorForTokenClassification(tokenizer)

# Define metrics and custom trainer

In [ ]:
import evaluate
import numpy as np

seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_preds, true_labels = [], []
    for pred, lab in zip(predictions, labels):
        temp_pred, temp_lab = [], []
        for p, l in zip(pred, lab):
            if l != -100:
                temp_pred.append(id2label[p])
                temp_lab.append(id2label[l])
        true_preds.append(temp_pred)
        true_labels.append(temp_lab)

    print("Sample predictions:", true_preds[:1])
    print("Sample labels:", true_labels[:1])
    return seqeval.compute(predictions=true_preds, references=true_labels, zero_division=0)

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits.view(-1, len(unique_labels)), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

# Save model

In [ ]:
model.save_pretrained("amharic-ner-model")
tokenizer.save_pretrained("amharic-ner-model")

In [ ]:
import shutil

shutil.make_archive("amharic-ner-model", 'zip', "amharic-ner-model")


# Download the .zip File

In [ ]:
from IPython.display import FileLink
FileLink('amharic-ner-model.zip')